# 📸 Image to PDF Converter

Convert images to PDF files easily! Supports:
- Single images
- Multiple images (creates ZIP file)
- ZIP files containing images
- Folders with images

**Supported formats:** PNG, JPG, JPEG, WEBP


## 🚀 How to Use

**Option 1:** Set `path_arg` to your file/folder path
**Option 2:** Upload files directly (in Colab)

### Examples:
```python
# Single image
path_arg = "photo.jpg"

# Folder with images  
path_arg = "/path/to/images/"

# ZIP file
path_arg = "images.zip"
```


In [ ]:
import os, zipfile, shutil, uuid
from PIL import Image

try:
    from google.colab import files
    IS_COLAB = True
    print("🔧 Running in Google Colab")
except ImportError:
    IS_COLAB = False
    print("🔧 Running locally")

In [ ]:
def get_images(path):
    if not os.path.exists(path):
        raise FileNotFoundError(f"❌ Path not found: {path}")
    
    if zipfile.is_zipfile(path):
        print(f"📦 Extracting ZIP: {os.path.basename(path)}")
        base = os.path.splitext(os.path.basename(path))[0]
        tmp_dir = f"ext_{uuid.uuid4().hex[:6]}"
        os.makedirs(tmp_dir, exist_ok=True)
        zipfile.ZipFile(path).extractall(tmp_dir)
        imgs = [os.path.join(r, f) for r, _, fs in os.walk(tmp_dir) 
                for f in fs if f.lower().endswith(('png','jpg','jpeg','webp'))]
        return imgs, base, tmp_dir
    elif os.path.isdir(path):
        print(f"📁 Scanning folder: {os.path.basename(path)}")
        base = os.path.basename(os.path.normpath(path))
        imgs = [os.path.join(r, f) for r, _, fs in os.walk(path) 
                for f in fs if f.lower().endswith(('png','jpg','jpeg','webp'))]
        return imgs, base, None
    else:
        print(f"🖼️ Processing single image: {os.path.basename(path)}")
        return [path], os.path.splitext(os.path.basename(path))[0], None


In [ ]:
def process(path):
    try:
        imgs, base, tmp = get_images(path)
        if not imgs:
            print("❌ No valid images found.")
            return

        print(f"📊 Found {len(imgs)} image(s)")
        
        out_dir = f"pdfs_{uuid.uuid4().hex[:6]}"
        os.makedirs(out_dir, exist_ok=True)
        
        pdf_paths = []
        for i, img in enumerate(imgs, 1):
            print(f"🔄 Converting {i}/{len(imgs)}: {os.path.basename(img)}")
            pdf_path = os.path.join(out_dir, os.path.splitext(os.path.basename(img))[0] + ".pdf")
            Image.open(img).convert("RGB").save(pdf_path)
            pdf_paths.append(pdf_path)

        if len(pdf_paths) == 1:
            final_file = f"{base}.pdf"
            shutil.move(pdf_paths[0], final_file)
            print(f"✅ Created: {final_file}")
        else:
            final_file = f"{base}_pdfs.zip"
            with zipfile.ZipFile(final_file, 'w', zipfile.ZIP_DEFLATED) as z:
                [z.write(p, os.path.basename(p)) for p in pdf_paths]
            print(f"✅ Created ZIP: {final_file} ({len(pdf_paths)} PDFs)")

        if IS_COLAB:
            files.download(final_file)
            print("📥 Download started!")
        else:
            print(f"📁 Output: {os.path.abspath(final_file)}")

    except Exception as e:
        print(f"❌ Error: {e}")
    finally:
        shutil.rmtree(out_dir, ignore_errors=True)
        if tmp: shutil.rmtree(tmp, ignore_errors=True)


## ⚙️ Configuration

Set your file path here:


In [ ]:
path_arg = None  # Set your file/folder path here


## 🎯 Run Conversion

Execute the conversion process:


In [ ]:
if path_arg:
    print(f"🚀 Processing: {path_arg}")
    process(path_arg)
elif IS_COLAB:
    print("📤 Upload your files...")
    uploaded = files.upload()
    for fname in uploaded.keys():
        print(f"🚀 Processing: {fname}")
        process(fname)
else:
    print("❗ Please set path_arg or upload in Colab.")
